In [1]:
import numpy as np
from numpy import asarray, array, ndarray
from scipy.signal import convolve
import math

In [2]:
randn = np.random.randn
import timeit
def time(x, h, mode='valid', method='auto', repeat=20, number=2):
    times = timeit.repeat(
        "convolve(x, h, mode='{}', method='{}')".format(mode, method), 
        "import numpy as np\n" +
        "from scipy.signal import convolve",
        repeat=repeat,
        number=number,
        globals={"x": x, "h": h}
    )
    return min(times) / number

In [11]:
from sklearn.utils import check_random_state
import pandas as pd
from scipy.stats import reciprocal as loguniform

def _get_data(a, b, ndim, rng):
    if ndim == 2:
        s1 = loguniform(min(3, a//2), a*2).rvs(random_state=rng).astype(int)
        s2 = loguniform(min(3, b//2), b*2).rvs(random_state=rng).astype(int)
        x = rng.randn(a, s1)
        h = rng.randn(b, s2)
        if mode == "valid":
            s = np.sort([a, b, s1, s2])
            h = rng.randn(*s[:2])
            x = rng.randn(*s[2:])
            assert all(h.shape[i] <= x.shape[j] for i in [0, 1] for j in [0, 1])
    elif ndim == 1:
        x = rng.randn(a)
        h = rng.randn(b)
    else:
        raise ValueError("ndim")
    return x, h

def time_conv(n, a, mode="full", ndim=2, random_state=None):
    rng = check_random_state(random_state)
    x, h = _get_data(n, a, ndim, rng)
        
    t_fft = time(x, h, mode=mode, method="fft")
    t_direct = time(x, h, mode=mode, method="direct")
    return {
        "fft_time": t_fft,
        "direct_time": t_direct,
        "mode": mode,
        "x.shape[0]": len(x),
        "h.shape[0]": len(h),
        "random_state": random_state
    }

In [12]:
import itertools
# sizes = np.logspace(np.log10(3), np.log10(500), num=100).astype("int")
# sizes = np.unique(sizes)
sizes = np.logspace(np.log10(100), np.log10(100e3), num=40).astype("int")
sizes = np.unique(sizes)
print(sizes)
today = "2019-11-03"

data = []

[   100    119    142    170    203    242    289    345    412    492
    587    701    837   1000   1193   1425   1701   2030   2424   2894
   3455   4124   4923   5878   7017   8376  10000  11937  14251  17012
  20309  24244  28942  34551  41246  49238  58780  70170  83767 100000]


In [13]:
len(data)

0

In [15]:
fname = f"out/{today}-1d-train.parquet" 
for n, a, mode in itertools.product(
    sizes, sizes, ["full", "valid", "same"],
):
    datum = time_conv(n, a, random_state=len(data), ndim=1, mode=mode)
    data.append(datum)
    if len(data) % 100 == 0:
        print(n, a, mode, len(data))
    if len(data) % 10 == 0:
        pd.DataFrame(data).to_parquet(fname, index=False)

100 11937 full 300
119 3455 valid 400
142 1000 same 500
170 345 full 600
203 100 valid 700
203 34551 same 800
242 11937 full 900
289 3455 valid 1000
345 1000 same 1100
412 345 full 1200
492 100 valid 1300
492 34551 same 1400
587 11937 full 1500
701 3455 valid 1600
837 1000 same 1700
1000 345 full 1800
1193 100 valid 1900
1193 34551 same 2000
1425 11937 full 2100
1701 3455 valid 2200
2030 1000 same 2300
2424 345 full 2400
2894 100 valid 2500
2894 34551 same 2600
3455 11937 full 2700
4124 3455 valid 2800


KeyboardInterrupt: 